# Model Serving Function

In [1]:
import mlrun
import os

In [2]:
%nuclio cmd -c pip install lightgbm

In [3]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [4]:
import numpy as np
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [5]:
# nuclio: end-code

## Deploy and Test The Function

If you would like to change the location of the source data, set the `SAMPLE_DATA_SOURCE_URL_PREFIX` environment variable.

For example, set it to `/v3io/projects/demos-data/iguazio/`

In [6]:
# Set the source-data URL
url_prefix = os.environ.get('SAMPLE_DATA_SOURCE_URL_PREFIX', 'https://s3.wasabisys.com/iguazio/')

In [7]:
models_path = f'{url_prefix.rstrip("/")}/models/lightgbm/SampleModel.pkl'

In [8]:
fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'edmondg', 'framework': 'lightgbm'},
                            code_output='.')
fn.spec.default_class = 'LGBMModel'

In [9]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [10]:
# deploy the function
fn.apply(mlrun.platforms.auto_mount())
address = fn.deploy()

> 2021-05-27 10:59:36,778 [info] Starting remote function deploy
2021-05-27 10:59:36  (info) Deploying function
2021-05-27 10:59:36  (info) Building
2021-05-27 10:59:36  (info) Staging files and preparing base images
2021-05-27 10:59:36  (info) Building processor image
2021-05-27 10:59:45  (info) Build complete
2021-05-27 10:59:53  (info) Function deploy complete
> 2021-05-27 10:59:54,247 [info] function deployed, address=default-tenant.app.achikar-dev.iguazio-cd1.com:31534


In [11]:
# test the function
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

{'id': '47ae98cd-80c3-477b-aa73-2faef9a1b244',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}